# Modeling

This notebook's objective is to train models, doing hyperparameter tunning with optuna, and saving the best models in a folder for later evaluation 

In [2]:
import xgboost
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import pandas as pd
import optuna

In [6]:
data = pd.read_csv("../data/train_preprocessed.csv")

X = data.drop(columns=["fraud_label","user_id","transaction_id"])
y = data.fraud_label

For this project we are using optuna for the hyperparameter optimization. Optuna to make the hyperparameter optimization requieres an objective function, this function returns a metric which will be the one that the model will be trying to minimize or maximize, in this case, maximize, as we will try to maximize the recall metric in order to get the best results.

## Random Forest

In [ ]:
def objective(trial:optuna.Trial, X, y):

    params = {
        "n_estimators": trial.suggest_int('n_estimators', 100, 1000),
        "min_samples_split": trial.suggest_int('min_samples_split', 2, 10),
        "min_samples_leaf": trial.suggest_int('min_samples_leaf', 1, 5),
        "max_depth": trial.suggest_int('max_depth', 4, 30),
        "random_state": 42,
        "class_weight": "balanced"
    }

    rf = RandomForestClassifier(**params)
    score = cross_val_score(rf,X,y,cv=5,scoring='recall')
    recall = score.mean()
    return recall

In [8]:
study = optuna.create_study(direction='maximize')

# 2. Run the optimization
study.optimize(lambda trial: objective(trial, X, y), n_trials=50)
# 3. Results
print(f"Best trial: {study.best_trial.number}")
print(f"Best recall: {study.best_value:.4f}")
print(f"Best parameters: {study.best_params}")

[I 2026-02-26 12:55:59,289] A new study created in memory with name: no-name-e70f0c82-e3a8-4d1b-a4ba-daded672b7f3
[I 2026-02-26 12:56:03,658] Trial 0 finished with value: 0.4065934065934066 and parameters: {'n_estimators': 220, 'min_samples_split': 7, 'min_samples_leaf': 3, 'max_depth': 7}. Best is trial 0 with value: 0.4065934065934066.


[0.26373626 0.40659341 0.36263736 0.34065934 0.65934066]


[I 2026-02-26 12:56:30,826] Trial 1 finished with value: 0.27252747252747256 and parameters: {'n_estimators': 956, 'min_samples_split': 4, 'min_samples_leaf': 1, 'max_depth': 29}. Best is trial 0 with value: 0.4065934065934066.


[0.06593407 0.23076923 0.26373626 0.2967033  0.50549451]


[I 2026-02-26 12:56:35,808] Trial 2 finished with value: 0.27252747252747256 and parameters: {'n_estimators': 180, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_depth': 19}. Best is trial 0 with value: 0.4065934065934066.


...

[0.06593407 0.23076923 0.26373626 0.2967033  0.50549451]


[I 2026-02-26 13:04:44,584] Trial 48 finished with value: 0.44835164835164837 and parameters: {'n_estimators': 132, 'min_samples_split': 8, 'min_samples_leaf': 3, 'max_depth': 5}. Best is trial 46 with value: 0.45274725274725275.


[0.23076923 0.51648352 0.42857143 0.37362637 0.69230769]


[I 2026-02-26 13:04:46,626] Trial 49 finished with value: 0.43296703296703304 and parameters: {'n_estimators': 135, 'min_samples_split': 8, 'min_samples_leaf': 2, 'max_depth': 4}. Best is trial 46 with value: 0.45274725274725275.


[0.15384615 0.51648352 0.45054945 0.36263736 0.68131868]
Best trial: 46
Best recall: 0.4527
Best parameters: {'n_estimators': 146, 'min_samples_split': 9, 'min_samples_leaf': 3, 'max_depth': 5}


## XGBoost